In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from isce2_topsapp import download_slcs, download_orbits, download_dem_for_isce2, download_aux_cal
import json

# Input Data

In [3]:
data = {
        "reference_scenes": ["S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9"],
        "secondary_scenes": ["S1B_IW_SLC__1SDV_20210711T014922_20210711T014949_027740_034F80_859D",
                             "S1B_IW_SLC__1SDV_20210711T014947_20210711T015013_027740_034F80_D404",
                             "S1B_IW_SLC__1SDV_20210711T015011_20210711T015038_027740_034F80_376C"]
        }
reference_scenes = data['reference_scenes']
secondary_scenes = data['secondary_scenes']

In [4]:
dry_run = True

# SLC

Do everything but download SLCs

In [5]:
out_slc = download_slcs(reference_scenes,
                        secondary_scenes,
                        dry_run=dry_run)
out_slc

100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.25it/s]


{'ref_paths': ['S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9.zip'],
 'sec_paths': ['S1B_IW_SLC__1SDV_20210711T015011_20210711T015038_027740_034F80_376C.zip',
  'S1B_IW_SLC__1SDV_20210711T014947_20210711T015013_027740_034F80_D404.zip',
  'S1B_IW_SLC__1SDV_20210711T014922_20210711T014949_027740_034F80_859D.zip'],
 'extent': [-119.080994, 33.405682, -115.988029, 35.43090952856198]}

# Orbits

In [10]:
out_orbit = download_orbits(reference_scenes,
                            secondary_scenes,
                            dry_run=dry_run)
out_orbit

{'ref_orbit': PosixPath('orbits/S1B_OPER_AUX_POEORB_OPOD_20210812T111941_V20210722T225942_20210724T005942.EOF'),
 'sec_orbit': PosixPath('orbits/S1B_OPER_AUX_POEORB_OPOD_20210731T111940_V20210710T225942_20210712T005942.EOF')}

# DEM

Don't currently have "dry-run" for this localization.

In [7]:
out_dem = download_dem_for_isce2(out_slc['extent'])
out_dem

/Users/cmarshak/opt/anaconda3/envs/topsapp_env/lib/python3.8/site-packages/dem_stitcher/stitcher.py:125: UserWarning: . exists; data will be overwritten
  warnings.warn(f'{dest_dir} exists; data will be overwritten')


This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /Users/cmarshak/bekaert-team/hyp3-topsApp/tests/low_res.dem.wgs84
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /Users/cmarshak/bekaert-team/hyp3-to

{'extent_buffered': [-119.085, 33.402, -115.984, 35.435],
 'full_res_dem_path': PosixPath('full_res.dem.wgs84'),
 'low_res_dem_path': PosixPath('low_res.dem.wgs84')}

# Aux Cal

In [8]:
out_aux_cal = download_aux_cal()

# Combining Metadata

In [9]:
out = {**out_slc,
       **out_dem,
       **out_aux_cal,
       **out_orbit}
out

{'ref_paths': ['S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9.zip'],
 'sec_paths': ['S1B_IW_SLC__1SDV_20210711T015011_20210711T015038_027740_034F80_376C.zip',
  'S1B_IW_SLC__1SDV_20210711T014947_20210711T015013_027740_034F80_D404.zip',
  'S1B_IW_SLC__1SDV_20210711T014922_20210711T014949_027740_034F80_859D.zip'],
 'extent': [-119.080994, 33.405682, -115.988029, 35.43090952856198],
 'extent_buffered': [-119.085, 33.402, -115.984, 35.435],
 'full_res_dem_path': PosixPath('full_res.dem.wgs84'),
 'low_res_dem_path': PosixPath('low_res.dem.wgs84'),
 'aux_cal_dir': 'aux_cal',
 'ref_orbit': PosixPath('orbits/S1B_OPER_AUX_POEORB_OPOD_20210812T111941_V20210722T225942_20210724T005942.EOF'),
 'sec_orbit': PosixPath('orbits/S1B_OPER_AUX_POEORB_OPOD_20210731T111940_V20210710T225942_20210712T005942.EOF')}